# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = 'T9.AQT87.873.653'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['PAD_START', "PASS", "1H", "X"]

# what's your bid?

In [4]:
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, -1, -1, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': '4H',
  'insta_score': 0.4037,
  'expected_score': 326,
  'adjustment': 20},
 {'call': '2H', 'insta_score': 0.5911, 'expected_score': -7, 'adjustment': 30}]

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(["PAD_START", "PASS", "1H", "X", "4H", "PASS", "PASS", "X"])
bid.bid

Bid not valid 3H insta_score: 0.9810969829559326


'PASS'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.0153}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['T9.AQT8x.8xx.xxx Q8xxxx..Txxx.Qxx Ax.J9xxxx.AQ.J98 KJx.Kx.KJ9x.AKTx']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS','3D'])
print(sample_hands.shape[0])
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

1
T9.AQT8x.8xx.xxx QJ8xx.xxx.xxx.9x Axxx.Kxx.A9x.Jxx Kx.J9.KQJT.AKQT8


How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['K9532.K8.J4.KJT4',
 'A.J963.Q9832.Q96',
 'QT87.A75.T65.832',
 'J64.QT42.AK7.A75']

In [10]:
bidder_bots = [BotBid([False, False], hand, models, -1, -1, sampler, False) for hand in hands]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS',
 'PASS',
 'PASS',
 '1C',
 'PASS',
 '1D',
 'PASS',
 '1H',
 'PASS',
 '3H',
 'PASS',
 '3N',
 'PASS',
 '4H',
 'PASS',
 'PASS',
 'X',
 'PASS',
 'PASS',
 'PASS']